In [1]:
from pydbsp.zset import ZSet, ZSetAddition
from pydbsp.stream import step_until_fixpoint
from typing import Tuple, List

from pydbsp.algorithms.datalog import (
    EDB,
    Fact,
    Program,
    Rule,
)

Edge = Tuple[int, int]
GraphZSet = ZSet[Edge]

def create_test_zset_graph(n: int) -> GraphZSet:
    return ZSet({(k, k + 1): 1 for k in range(n)})

def create_test_edb(n: int) -> EDB:
    test_graph = create_test_zset_graph(n)
    group: ZSetAddition[Fact] = ZSetAddition()

    test_edb = group.identity()
    for k, v in test_graph.items():
        test_edb.inner[("E", (k[0], k[1]))] = v

    return test_edb

def from_facts_into_zset(facts: List[Tuple[Fact, int]]) -> EDB:
    edb: EDB = ZSetAddition().identity()
    for fact, weight in facts:
        edb.inner[fact] = weight

    return edb


def from_rule_into_zset(rule: Rule, weight: int) -> Program:
    program: Program = ZSetAddition().identity()
    program.inner[rule] = weight

    return program



In [2]:
from pydbsp.algorithms.datalog import IncrementalDatalogWithNegation, Variable
from pydbsp.stream import Stream, StreamHandle
from pydbsp.stream.operators.linear import stream_elimination

edb_group: ZSetAddition[Fact] = ZSetAddition()
edb_stream = Stream(edb_group)
edb_stream_h = StreamHandle(lambda: edb_stream)

program_group: ZSetAddition[Rule]= ZSetAddition()
program_stream = Stream(program_group)
program_stream_h = StreamHandle(lambda: program_stream)

seed: Rule = (("T", (Variable("X"), Variable("Y"))), ("E", (Variable("X"), Variable("Y"))))
transitivity: Rule = (
    ("T", (Variable("X"), Variable("Z"))),
    ("T", (Variable("X"), Variable("Y"))),
    ("T", (Variable("Y"), Variable("Z"))),
)
node_left: Rule = (
    ("Node", (Variable("X"),)),
    ("E", (Variable("X"), Variable("Y")))
)
node_right: Rule = (
    ("Node", (Variable("Y"),)),
    ("E", (Variable("X"), Variable("Y")))
)
complement_transitivity: Rule = (
    ("not_T", (Variable("X"), Variable("Y"))),
    ("Node", (Variable("X"),)),
    ("Node", (Variable("Y"),)),
    ("!T", (Variable("X"), Variable("Y")))
)

reasoner = IncrementalDatalogWithNegation(edb_stream_h, program_stream_h, None)
edb_stream.send(from_facts_into_zset([(("E", (2, 3)), 1), (("E", (0, 2)), 1), (("E", (0, 1)), 1), (("R", (1, 2)), 1) , (("E", (3, 4)), 1)]))
program_stream.send(from_rule_into_zset(transitivity, 1))
program_stream.send(from_rule_into_zset(node_left, 1))
program_stream.send(from_rule_into_zset(node_right, 1))
program_stream.send(from_rule_into_zset(complement_transitivity, 1))
program_stream.send(from_rule_into_zset(seed, 1))

step_until_fixpoint(reasoner)
fixedpoint = [ k[1] for k, _v in stream_elimination(reasoner.output()).items() if k[0] == "not_T" ]

print(sorted(fixedpoint))

[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (3, 2), (3, 3), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]


In [3]:
edb_group: ZSetAddition[Fact] = ZSetAddition()
edb_stream = Stream(edb_group)
edb_stream_h = StreamHandle(lambda: edb_stream)

program_group: ZSetAddition[Rule]= ZSetAddition()
program_stream = Stream(program_group)
program_stream_h = StreamHandle(lambda: program_stream)

cycle_4_not_overlapping_cycle_3: Rule = (
    ("cycle_4_not_overlapping_cycle_3", (
        Variable("A"), 
        Variable("B"), 
        Variable("C"), 
        Variable("D")
    )),
    ("E", (Variable("A"), Variable("B"))),
    ("E", (Variable("B"), Variable("C"))),
    ("!E", (Variable("A"), Variable("C"))),
    ("E", (Variable("C"), Variable("D"))),
    ("!E", (Variable("B"), Variable("D"))),
    ("!E", (Variable("C"), Variable("A"))),
    ("E", (Variable("D"), Variable("A"))),
    ("!E", (Variable("D"), Variable("B")))
)

reasoner = IncrementalDatalogWithNegation(edb_stream_h, program_stream_h, None)
edb_stream.send(from_facts_into_zset([
    #// First 4-cycle: 0 -> 1 -> 2 -> 3 -> 0
    (("E", (0, 1)), 1),
    (("E", (1, 2)), 1),
    (("E", (2, 3)), 1),
    (("E", (3, 0)), 1),
    
    #// Second pattern (4-cycle with 3-cycle): 4 -> 5 -> 6 -> 7 -> 4 and 6 -> 4
    (("E", (4, 5)), 1),
    (("E", (5, 6)), 1),
    (("E", (6, 7)), 1),
    (("E", (7, 4)), 1),
    (("E", (6, 4)), 1)  #// Creates the 3-cycle
]))
program_stream.send(from_rule_into_zset(cycle_4_not_overlapping_cycle_3, 1))

step_until_fixpoint(reasoner)
fixedpoint = [ k[1] for k, v in stream_elimination(reasoner.output()).items() if k[0] == "cycle_4_not_overlapping_cycle_3" ]

print(sorted(fixedpoint))

[(0, 1, 2, 3), (1, 2, 3, 0), (2, 3, 0, 1), (3, 0, 1, 2)]
